In [1]:
import pandas as pd
import sqlite3
 
con = sqlite3.connect('Db-IMDB.db')

# 1.List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [2]:
query1 = """
select t9.Name,t9.title,t9.year from 
(SELECT t8.Name,t7.title,t7.year, row_number() over(partition by t7.MID) as rn FROM
(SELECT t5.MID,t5.title,t5.year,t6.PID FROM
(select t3.MID,t3.title,t3.year from
(select t1.MID,t1.title,SUBSTR(t1.year, LENGTH(t1.year) - 3, 4) as year, t2.GID 
from movie as t1 left join M_Genre as t2 on t1.MID = t2.MID 
where (SUBSTR(t1.year, LENGTH(t1.year) - 3, 4) % 4 = 0 and 
SUBSTR(t1.year, LENGTH(t1.year) - 3, 4) % 100 <> 0)
or (SUBSTR(t1.year, LENGTH(t1.year) - 3, 4) % 400 = 0))t3 
left join Genre as t4
on t3.GID = t4.GID
where t4.Name like '%Comedy%')t5
left join M_Director as t6
on t5.MID = t6.MID)t7
left join Person as t8
on t7.PID = t8.PID)t9
where rn = 1
order by t9.year desc
"""

result1 = pd.read_sql(query1, con)

result1.head(10)

,Name,title,year
0,Akashdeep,Santa Banta Pvt Ltd,2016
1,Rohit Dhawan,Dishoom,2016
2,Milap Zaveri,Mastizaade,2016
3,Sunil Subramani,Fuddu,2016
4,Rajiv S. Ruia,Direct Ishq,2016
5,Bobby Kolli,Sardaar Gabbar Singh,2016
6,Sajid,Housefull 3,2016
7,Indra Kumar,Great Grand Masti,2016
8,R. Balki,Ki & Ka,2016
9,Shakun Batra,Kapoor & Sons,2016


# 2.List the names of all the actors who played in the movie 'Anand' (1971)

In [3]:
query2 = """
select t3.title, t3.name from  
(select * from  
(select * from
(select ID, MID, TRIM(PID) as PID1 from M_Cast)t1
JOIN Movie m ON m.MID=t1.MID where m.title='Anand')t2
JOIN Person p ON p.PID=t2.PID1)t3

"""

result2 = pd.read_sql(query2, con)

result2

,title,Name
0,Anand,Amitabh Bachchan
1,Anand,Rajesh Khanna
2,Anand,Brahm Bhardwaj
3,Anand,Ramesh Deo
4,Anand,Seema Deo
5,Anand,Dev Kishan
6,Anand,Durga Khote
7,Anand,Lalita Kumari
8,Anand,Lalita Pawar
9,Anand,Atam Prakash


# 3.List all the actors who acted in a film before 1970 andin a film after 1990. (That is: < 1970 and > 1990.)

In [4]:
query3 = """
select distinct ta.Name from
(select t2.year, trim(p.Name) as Name from
(select TRIM(PID) as PID1,t1.title,t1.year from
(select MID,title,year from movie where LENGTH(year)=4 AND year > 1990)t1
JOIN M_Cast c ON c.MID=t1.MID)t2
JOIN Person p ON p.PID=t2.PID1)ta inner join
(select t2.year, trim(p.Name) as Name from
(select TRIM(PID) as PID1,t1.title,t1.year from
(select MID,title,year from movie where LENGTH(year)=4 AND year < 1970)t1
JOIN M_Cast c ON c.MID=t1.MID)t2
JOIN Person p ON p.PID=t2.PID1)tb
on trim(ta.Name) = trim(tb.Name) 
"""

result3 = pd.read_sql(query3,con)
result3

,Name
0,Rishi Kapoor
1,Anand Tiwari
2,Amitabh Bachchan
3,Rajesh Kumar
4,Asrani
...,...
412,Kishan Mehta
413,Vijayalaxmi
414,Manohar Deepak
415,Brij Bhushan


# 4.List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [5]:
query4 = """
with main as (select * from
(select count(MID) as ct, PID 
from M_Director group by PID)t1
where t1.ct>9 order by t1.ct desc)

select t2.Name, m.ct from
(select * from
(select *, row_number() over(partition by p.PID) as rn 
from Person p)t1
where t1.rn = 1)t2 inner join main m on m.PID=t2.PID
"""

result4 = pd.read_sql(query4,con)

result4.head(10)

,Name,ct
0,David Dhawan,39
1,Mahesh Bhatt,35
2,Priyadarshan,30
3,Ram Gopal Varma,30
4,Vikram Bhatt,29
5,Hrishikesh Mukherjee,27
6,Yash Chopra,21
7,Basu Chatterjee,19
8,Shakti Samanta,19
9,Subhash Ghai,18


# 5.a.For each year, count the number of movies in that year that had only female actors.b.Now include a small change: report for each year the percentage of movies in that year with only female actors, andthe total number of movies made that year. For example, one answer will be:1990 31.81 13522meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [6]:
query5a = """ 
select tx.year, count(tx.title) as movies_in_year from
(select * from
(select count(t3.gender) as gen_cnt,t3.title,gender,
substr(t3.year,length(t3.year)-3,4) as year from  
(select * from  
(select * from
(select ID, MID, TRIM(PID) as PID1 from M_Cast)t1
JOIN Movie m ON m.MID=t1.MID)t2
JOIN Person p ON p.PID=t2.PID1)t3
where t3.gender = 'Female'  
group by t3.title,t3.gender)ta
inner join
(select  title, sum(gen_cnt) as total_cnt,year from
(select count(t3.gender) as gen_cnt,t3.title,gender,year from  
(select * from  
(select * from
(select ID, MID, TRIM(PID) as PID1 from M_Cast)t1
JOIN Movie m ON m.MID=t1.MID)t2
JOIN Person p ON p.PID=t2.PID1)t3  
group by t3.title,t3.gender)t4
group by title)tb
on ta.title = tb.title
where ta.gen_cnt = tb.total_cnt)tx
group by tx.year
"""

result5a = pd.read_sql(query5a, con)
result5a.head()

,year,movies_in_year
0,1939,1
1,1999,1
2,2000,1
3,2018,1


In [7]:
query5b = """
select ac.year, ac.total_movies, 
cast((cast(ac.movies_in_year as float)*100)/ac.total_movies as float) as percentage
from (select * from
(select count(t3.title) as total_movies, t3.year1 from
(select *, substr(t2.year,length(t2.year)-3,4) as year1 from  
(select * from
(select ID, MID, TRIM(PID) as PID1 from M_Cast)t1
JOIN Movie m ON m.MID=t1.MID)t2
JOIN Person p ON p.PID=t2.PID1)t3  
group by t3.year1)tz
inner join
(select tx.year, count(tx.title) as movies_in_year from
(select * from
(select count(t3.gender) as gen_cnt,t3.title,gender,
substr(t3.year,length(t3.year)-3,4) as year from  
(select * from  
(select * from
(select ID, MID, TRIM(PID) as PID1 from M_Cast)t1
JOIN Movie m ON m.MID=t1.MID)t2
JOIN Person p ON p.PID=t2.PID1)t3
where t3.gender = 'Female'  
group by t3.title,t3.gender)ta
inner join
(select  title, sum(gen_cnt) as total_cnt,year from
(select count(t3.gender) as gen_cnt,t3.title,gender,year from  
(select * from  
(select * from
(select ID, MID, TRIM(PID) as PID1 from M_Cast)t1
JOIN Movie m ON m.MID=t1.MID)t2
JOIN Person p ON p.PID=t2.PID1)t3  
group by t3.title,t3.gender)t4
group by title)tb
on ta.title = tb.title
where ta.gen_cnt = tb.total_cnt)tx
group by tx.year)ty
on tz.year1 = ty.year)ac

"""
result5b = pd.read_sql(query5b,con)
result5b.head(10)

,year,total_movies,percentage
0,1939,46,2.173913
1,1999,1709,0.058514
2,2000,1578,0.063371
3,2018,3009,0.033234


# 6.Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts,we still count her/him only once

In [8]:
query6 = """
select t4.title, max(t4.cast_count) as max_cast_count from
(select t3.title, count(t3.PID1) as cast_count from
(select distinct t2.title, t2.PID1 from
(select * from
(select ID, MID, TRIM(PID) as PID1 from M_Cast)t1
JOIN Movie m ON m.MID=t1.MID)t2)t3
group by t3.title)t4

"""

result6 = pd.read_sql(query6,con)
result6.head(10)

,title,max_cast_count
0,Ocean's Eight,238


# 7.A decade is a sequence of 10 consecutive years. For example,say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [9]:
#reference - https://stackoverflow.com/questions/51609285/query-for-find-the-decade-with-the-largest-number-of-records---(non equi joins) 
query7 = """
select t1.year as start,
t1.year + 9 as end, count(*) as year_count from 
(select distinct year from Movie)t1 
join Movie m
on m.year >= t1.year and m.year < t1.year + 10
group by t1.year order by count(*) desc limit 1
"""

result7 = pd.read_sql(query7,con)
result7.head()

,start,end,year_count
0,2008,2017,1128


# 8.Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies).

In [10]:
query8 = """

select p.Name,P.Gender, t4.year1, t4.next_working_year, t4.diff from
(select t3.PID, t3.year1, t3.next_working_year, MAX(t3.year1 - t3.next_working_year) as diff from
(select Trim(t1.PID) as PID, t1.year1,t1.next_working_year, count(t1.MID) from
(select m.MID, mc.PID, substr(m.year,length(m.year)-3,4) as year1,
lead(substr(m.year,length(m.year)-3,4),1) 
over(partition by mc.PID order by substr(m.year,length(m.year)-3,4) desc) as next_working_year
from M_Cast mc join Movie m on mc.MID = m.MID order by substr(m.year,length(m.year)-3,4) desc)t1
group by t1.PID,t1.year1 having t1.next_working_year is not null)t3 
group by t3.PID having MAX(t3.year1 - t3.next_working_year) < 4  and MAX(t3.year1 - t3.next_working_year) <> 0) t4
join Person p on p.PID = t4.PID

"""

result8 = pd.read_sql(query8,con)
result8.head()

,Name,Gender,year1,next_working_year,diff
0,Gillian Anderson,Female,2017,2016,1
1,Shabana Azmi,Female,1988,1986,2
2,Amitabh Bachchan,Male,1986,1985,1
3,Sarita Choudhury,Female,2011,2008,3
4,Jayant,Male,1958,1955,3


# 9.Find all the actors that made more movies with Yash Chopra than any other director.

In [11]:
query9 = """with 
not_yc_movies as 
(select a_PID, d_PID, mc_MID , max(movies_not_yc) as max_mov_not_yc from
(select * , count(md_MID) as movies_not_yc from
(select t1.*, trim(md.MID) as md_MID, trim(md.PID) as d_PID from
(select trim(MID) as mc_MID, TRIM(PID) as a_PID from M_Cast)t1
join M_Director md on t1.mc_MID = trim(md.MID))t2 group by a_PID, d_PID
having d_PID <>  "nm0007181")t2 group by a_PID),
yc_movies as
(select a_PID, d_PID, mc_MID , count(md_MID) as movies_yc from
(select t1.*, trim(md.MID) as md_MID, trim(md.PID) as d_PID from
(select trim(MID) as mc_MID, TRIM(PID) as a_PID from M_Cast)t1
join M_Director md on t1.mc_MID = trim(md.MID))t2 group by a_PID, d_PID
having d_PID =  "nm0007181")

select table3.Name, table3.movies_yc, table3.cp as max_movies_without_yc from 
(select trim(p.Name) as Name, table2.a_PID, table2.movies_yc, table2.cp, row_number() over(partition by trim(p.Name)) as rn from (select * from
(select ta.*, tb.max_mov_not_yc, (CASE WHEN tb.max_mov_not_yc is null THEN 0 ELSE tb.max_mov_not_yc END)
as cp from yc_movies ta left join not_yc_movies tb on ta.a_PID = tb.a_PID)table1
where movies_yc > cp)table2 left join Person p on trim(p.PID) = table2.a_PID)table3
where rn = 1 order by table3.movies_yc desc

"""
result9 = pd.read_sql(query9,con)
result9.head(20) 

,Name,movies_yc,max_movies_without_yc
0,Jagdish Raj,11,6
1,Manmohan Krishna,10,4
2,Iftekhar,9,4
3,Shashi Kapoor,7,3
4,Rakhee Gulzar,5,3
5,Waheeda Rehman,5,3
6,Achala Sachdev,4,3
7,Neetu Singh,4,3
8,Leela Chitnis,3,2
9,Sudha Chopra,3,2


# 10.The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whoseShahrukh number is 2.

In [12]:
query10 = """
with M_Cast1 as (select trim(PID) as PID, trim(MID) as MID from M_Cast),
Person1 as (select * , trim(PID) as PID1, trim(Name) as Name1 from Person)

select distinct p.Name1 from
(select distinct t2.PID1, t2.MID1 as MID_with_srk , mc1.MID as MID_without_srk from
(select mc.PID as PID1, mc.MID as MID1 from
(select MID as MID, PID as s_PID from M_Cast1 where PID = "nm0451321")t1 
join M_Cast1 mc on mc.MID = t1.MID where mc.PID <> "nm0451321")t2 join M_Cast1 mc1 on
mc1.PID = t2.PID1 and mc1.MID <> t2.MID1)t3 join M_Cast1 mc2 
on t3.MID_without_srk = mc2.MID join Person1 p  on mc2.PID = p.PID1 where p.PID not in (t3.PID1)

"""
result10 = pd.read_sql(query10,con)
result10.head(20)

,Name1
0,Alicia Vikander
1,Dominic West
2,Walton Goggins
3,Daniel Wu
4,Kristin Scott Thomas
5,Derek Jacobi
6,Alexandre Willaume
7,Tamer Burjaq
8,Adrian Collins
9,Keenan Arrison
